### 装饰器

需要强调的是装饰器并不会修改原始函数的参数签名以及返回值。 使用 `*args` 和 `**kwargs` 目的就是确保任何参数都能适用。 而返回结果值基本都是调用原始函数`func(*args, **kwargs)`的返回结果，其中func就是原始函数。

@wraps(func) 注解是很重要的， 它能保留原始函数的元数据

wraps 实际上是另外一个装饰器，作用是，用func的元数据替换wrapper的元数据

```
# 不加@wraps(func)
help(countdown) 
# wrapper(*args, **kwargs)

# 加@wraps(func)
help(countdown) 
# countdown(n)
#    Counts down
```

有多个包装器，那么访问 __wrapped__ 属性的行为是不可预知的，应该避免这样做

并不是所有的装饰器都使用了 @wraps，特别的，内置的装饰器 @staticmethod 和 @classmethod 就没有遵循这个约定 (它们把原始函数存储在属性 __func__ 中）

In [ ]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result

    return wrapper

@timethis
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

countdown(1000000)

In [ ]:
from inspect import signature

print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)
print(signature(countdown))

In [ ]:
# 直接访问被包装函数
countdown.__wrapped__(100000)

In [ ]:
# @staticmethod, @classmethod, @property实际都是装饰器
# 同时，作为类属性的他们，底层是由描述器实现的
class A:
    @classmethod
    def method(cls):
        pass

class B:
    # Equivalent definition of a class method
    def method(cls):
        pass
    method = classmethod(method)

In [85]:
# 纯描述器实现
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

func = countdown
def wrapper(*args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    print(func.__name__, end-start)
    return result

wraps(countdown)(wrapper)
countdown = wrapper

countdown(1000000)

countdown 0.05351901054382324


### 带参数的装饰器

实际上使用闭包，把外部参数传进去

注意这里区别，一个带括号，一个不带括号


> @timethis
> 
> @logged(logging.DEBUG)


In [ ]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        return wrapper

    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

### 带外部变量的装饰器

访问函数 attach_wrapper 在进一步包装wrapper

等同于, 但如果它的上面还有另外的装饰器，那么它会隐藏底层属性，使得修改它们没有任何作用。

```
def set_level(newlevel):
    nonlocal level
    level = newlevel

wrapper.set_level = set_level
```

In [ ]:
from functools import wraps, partial
import logging
# Utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    # 注解中调用 attach_wrapper(wrapper)
    # 返回 partial(attach_wrapper, obj)
    if func is None:
        return partial(attach_wrapper, obj)
    # 实际调用时，带func可直接执行下面
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        # Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper

    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2, 3)

# Change the log message
add.set_message('Add called')
add(2, 3)

# Change the log level
add.set_level(logging.WARNING)
add(2, 3)

In [ ]:
# 访问函数会在多层装饰器间传播
# 但如果不是访问函数绑定，而是上面说的直接绑定，就无法传播了
@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1
        
countdown(10000000)
countdown.set_level(logging.WARNING)
countdown.set_message("Counting down to zero")
countdown(10000000)

### 可选参数的装饰器

这里使用小技巧使 @logged 和 @logged() 统一

加括号时，func=None, 调用下面的partial

> partial(logged, level=level, name=name, message=message)

实际使用时，会带着func所以直接执行后面的代码

In [ ]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__

    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)

    return wrapper

# Example use
@logged
def add(x, y):
    return x + y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')

### 装饰器+类型检查

用注解中的 type 对函数的签名进行绑定和检查

bound_types 通过注解指定的类型

bound_values 实际拿到的值的类型

In [ ]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func

        # Map function argument names to supplied types
        # 用注解中的 type 对函数的签名进行绑定
        sig = signature(func)
        # bind_partial 允许忽略参数
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @wraps(func)
        def wrapper(*args, **kwargs):
            # bind 不允许忽略参数
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                            )
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [ ]:
@typeassert(int, int)
def add(x, y):
    return x + y

add(2, 3)
add(2, 'hello')

In [ ]:
# 部分指定
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

spam(1, 2, 3)
spam(1, 'hello', 3)
spam(1, 'hello', 'world')

### 在类中定义装饰器

In [ ]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper

    # Decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper
    
# As an instance method
a = A()
@a.decorator1
def spam():
    pass

# As a class method
@A.decorator2
def grok():
    pass

In [27]:
# @property 装饰器实际上就是一个类，类似上面的 decorator1
class Person:
    # Create a property instance
    first_name = property()

    # Apply decorator methods
    @first_name.getter
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

### 将装饰器定义为类

确保它实现了 __call__() 和 __get__() 方法

`__get__`方法主要是确保底层描述器正常

In [181]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        # 将被包装函数的元信息复制到实例
        # 实际上此时实例的信息已经被替换为函数(add)的信息了
        wraps(func)(self)
        self.ncalls = 0

    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            # 将装饰器实例(self)绑定到外部实例(instance)上去
            return types.MethodType(self, instance) 

@Profiled
def add(x, y):
    return x + y

class Spam:
    @Profiled
    def bar(self, x):
        print(x)

In [182]:
# 等同于
# 底层实现还是描述器
# __get__方法主要是进行方法绑定
# 因为 spam.bar 中 bar 是描述器实例（类的属性）
# 而内部的 self.__wrapped__ 是依赖于实例的
# 用 MethodType 将描述器（类的属性）绑定到外部的实例上
# 否则，调用时候就要指明self实例，如 spam.bar(spam.bar, 3)
def add(x, y):
    return x + y

add = Profiled(add)
print(add(2, 3))
print(add.ncalls)

class Spam:
    def bar(self, x):
        print(x)
    
    bar = Profiled(bar)
    # 1. init 用 bar 的元信息装饰器实例的元信息
    # 2. 调用 spam.bar 时，在 get 方法中将装饰器实例绑定到外部实例上
    # 3. () 调用的是装饰器实例的 call

spam = Spam()
spam.bar(3)
print(spam.bar.ncalls)

5
1
3
1


In [203]:
class A:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

class B:
    def bar(self, x):
        print(x)

    a = A(bar)

b = B()
# print(b.a(1)) # 这里是从类B中找到的a，而不是从实例b中找到的a
b.a(b.a, 1) # 需要指明实例b

b.a = types.MethodType(b.a, b) # 把a绑定到实例b中
b.a(1)

1
1


In [ ]:
# 绑定方法和函数的区别

class Spam:
    pass

s = Spam()
def bar(self, x):
    print(x)

s.bar = bar
print(s.__dict__) # 函数
s.bar(s, 3) # 函数调用时要给 self

s.bar = types.MethodType(bar, s)
print(s.__dict__) # 绑定方法
s.bar(3) # 绑定方法调用时不用给 self

### 为类和静态方法提供装饰器

问题在于 @classmethod 和 @staticmethod 实际上并不会创建可直接调用的对象， 而是创建特殊的描述器对象。因此当你试着在其他装饰器中将它们当做函数来使用时就会出错。确保这种装饰器出现在装饰器链中的第一个位置可以修复这个问题。

@classmethod 和 @staticmethod 写在第一个位置

```
@classmethod
@timethis
```

```
@staticmethod
@timethis
```

```
@classmethod
@abstractmethod
```

### 装饰器增加参数

实际上使用 wrapper 替换函数，所以可以通过给 wrapper 增加强制关键字参数实现

In [205]:
from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)

    return wrapper

@optional_debug
def spam(a,b,c):
    print(a,b,c)
spam(1,2,3)
spam(1,2,3, debug=True)

1 2 3
Calling spam
1 2 3


In [ ]:
# 进一步优化函数签名
from functools import wraps
import inspect

def optional_debug(func):
    if 'debug' in inspect.getargspec(func).args:
        raise TypeError('debug argument already defined')

    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)

    sig = inspect.signature(func)
    parms = list(sig.parameters.values())
    parms.append(inspect.Parameter('debug',
                inspect.Parameter.KEYWORD_ONLY,
                default=False))
    wrapper.__signature__ = sig.replace(parameters=parms)
    return wrapper

### 类装饰器

类装饰器通常可以作为其他高级技术比如混入或元类的一种非常简洁的替代方案

In [210]:
def log_getattribute(cls):
    # Get the original implementation
    orig_getattribute = cls.__getattribute__

    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)

    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls

# Example use
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x

    def spam(self):
        pass
    
a = A(42)
a.x

getting: x


42

In [212]:
### 继承实现
class LoggedGetattribute:
    def __getattribute__(self, name):
        print('getting:', name)
        return super().__getattribute__(name)

# Example:
class A(LoggedGetattribute):
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass

a = A(42)
a.x

getting: x


42

### 装饰器+内联回调函数

In [ ]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)

    # Invoke the callback with the result
    callback(result)

In [ ]:
from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

> f = func(*args)返回一个生成器，第一次调用f.send(None)运行到第一个yield位置拿到第一个参数，然后执行计算和callback塞入队列，之后再从队列里面拿出来，通过下一次send返回给当前的yield

In [ ]:
def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

test()

In [ ]:
# 替换为多进程方案
import multiprocessing
pool = multiprocessing.Pool()
apply_async = pool.apply_async

# Run the test function
test()

In [ ]:
import time, random

def add(x, y):
    time.sleep(random.randint(0,5))
    return x + y

# 替换为多进程方案
import multiprocessing
pool = multiprocessing.Pool()
apply_async = pool.apply_async

# Run the test function
test()